**Import Section**

Importing relavent packages for the project.

Note: We will ignore warnings for simplicity.

In [358]:
import numpy as py
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

**Creating initial dataframes**

Following files are available for the project:

1. movie_characters_metadata.tsv
2. movie_conversations.tsv
3. movie_lines.tsv
4. movie_titles_metadata.tsv
5. raw_script_urls


Here are the list of initial dataframes for the respective files:
1. characters_df
2. conversations_df
3. lines_df
4. titles_df
5. rawScript_df


General File Features:
1. Files are tab seperated ('\t').
2. There is no header or footers in any of the files.
3. Few bad rows are present in the files. These rows will be dropped while dataframe creation. 


In [427]:
##Creating characters_df dataframe from movie_characters_metadata.tsv

column_names = ['chId', 'chName', 'mId', 'mName', 'gender', 'posCredits']
characters_df = pd.read_csv('movie_characters_metadata.tsv', sep='\t', header=None, names=column_names, on_bad_lines='skip')
print("SUCESS : 'characters_df' dataframe created from 'movie_characters_metadata.tsv'\n")
print('Here is a snapshot of data')
characters_df.head()


SUCESS : 'characters_df' dataframe created from 'movie_characters_metadata.tsv'

Here is a snapshot of data


,chId,chName,mId,mName,gender,posCredits
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [416]:
print("Dimension & metadata of characters_df",characters_df.shape)
print(characters_df.info())

Dimension & metadata of characters_df (9034, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9034 entries, 0 to 9033
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   chId        9034 non-null   object
 1   chName      9015 non-null   object
 2   mId         9017 non-null   object
 3   mName       9017 non-null   object
 4   gender      9017 non-null   object
 5   posCredits  9017 non-null   object
dtypes: object(6)
memory usage: 423.6+ KB
None


In [423]:
##Checking on the percentage of NaN for each column in the dataframe

nan_percentage = (characters_df.isna().mean() * 100).round(2)
print("Distribution Of NaN across dataframe")
print(nan_percentage)

Distribution Of NaN across dataframe
chId          0.00
chName        0.21
mId           0.19
mName         0.19
gender        0.19
posCredits    0.19
dtype: float64


In [362]:
# Removing rows with NaN values
characters_df = characters_df.dropna()

In [363]:
# Remove duplicate rows
duplicate_counts = characters_df.duplicated().sum()
print("Total duplicate", duplicate_counts)
if duplicate_counts > 0:
  characters_df = characters_df.drop_duplicates()

Total duplicate 0


**Data Cleansing: Removing Ambiguity From The Column Gender**

In this project we are dividing gender to two groups "Male" (M) & "Female" (F). 

However during the initial data analysis, it is identified that 4 different types of values are available in the column gender('?', 'm', 'f', 'M'). On further investigation, it has been confirmed all gender where data is 'm' & 'M' belong to male class rest can be grouped as female.In this dataframe 'Male' class will be represented as 'M' & 'Female' class will be represented as 'F'.

In [364]:
##Count of values of each gender type
characters_df.gender.value_counts()

gender
?    6006
m    1899
f     921
M     145
F      44
Name: count, dtype: int64

In [425]:
##Removing data ambiguity from gender
characters_df = characters_df[characters_df.gender != '?']
characters_df.gender = characters_df.gender.apply(lambda g: 'M' if g in ['m', 'M'] else 'F')
print("Data Distribution Per Gender:")
characters_df.gender.value_counts()

Data Distribution Per Gender:


gender
M    2044
F     982
Name: count, dtype: int64

**Data Cleansing: Removing non-numeric values from the column posCredits**

As per the column defination, 'posCredits' is  numeric column containing the information of the position where the character features in the credits. Hence this cannot be non-numeric. 

As part of the cleansing strategy we will remove such rows as very such rows are available.

In [366]:
# Display the total count of non-numeric values
non_numeric_count = pd.to_numeric(characters_df['posCredits'], errors='coerce').isna().sum()
print("Total count of non-numeric values in 'posCredits' removed {}".format(non_numeric_count))

# Remove rows with non-numeric values in the specified column
characters_df = characters_df[pd.to_numeric(characters_df['posCredits'], errors='coerce').notna()]

Total count of non-numeric values in 'posCredits' removed 330


**Movie title Dataframe**

In [426]:
##Creating the dataframe : titles_df

column_names = ['mId', 'mName', 'mYear', 'mRating', 'mVotes', 'mGenre']
titles_df = pd.read_csv('movie_titles_metadata.tsv', sep='\t', names=column_names, header=None, on_bad_lines='skip')
print("SUCESS : 'titles_df' dataframe created from 'movie_titles_metadata.tsv'\n")
print('Here is a snapshot of data')
titles_df.head()

SUCESS : 'titles_df' dataframe created from 'movie_titles_metadata.tsv'

Here is a snapshot of data


,mId,mName,mYear,mRating,mVotes,mGenre
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227.0,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289.0,['action' 'comedy' 'crime' 'drama' 'thriller']


In [368]:
print(titles_df.shape)
print(titles_df.info())


(617, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   mId      617 non-null    object 
 1   mName    616 non-null    object 
 2   mYear    616 non-null    object 
 3   mRating  616 non-null    float64
 4   mVotes   616 non-null    float64
 5   mGenre   616 non-null    object 
dtypes: float64(2), object(4)
memory usage: 29.1+ KB
None


Removing rows with NAN

In [369]:
nan_percentage = (titles_df.isna().mean() * 100).round(2)
print(nan_percentage)

mId        0.00
mName      0.16
mYear      0.16
mRating    0.16
mVotes     0.16
mGenre     0.16
dtype: float64


In [370]:
# Remove rows with NaN values
titles_df = titles_df.dropna()

In [371]:
# Remove duplicate rows
duplicate_counts = titles_df.duplicated().sum()
print("Total Number of duplicate rows", duplicate_counts)
if duplicate_counts > 0:
  titles_df = titles_df.drop_duplicates()

Total Number of duplicate rows 0


No outier data found in numeric columns

In [372]:
titles_df.describe()

,mRating,mVotes
count,616.000000,616.000000
mean,6.865584,49901.698052
std,1.215463,61898.367352
min,2.500000,9.000000
25%,6.200000,9992.500000
50%,7.000000,27121.500000
75%,7.800000,66890.000000
max,9.300000,419312.000000


Extracting the year part from the column mYear

In [373]:
column_name = 'mYear' 

# Filter non-numeric values in the specified column
non_numeric_values = titles_df.loc[~pd.to_numeric(titles_df[column_name], errors='coerce').notna(), column_name]

# Get unique non-numeric values
unique_non_numeric_values = non_numeric_values.unique()

# Display the unique non-numeric values
print("Unique non-numeric values in column '{}':".format(column_name))
print(unique_non_numeric_values)

Unique non-numeric values in column 'mYear':
['1989/I' '1990/I' '1995/I' '1998/I' '2004/I' '2007/I' '1992/I' '2005/I'
 '2002/I' '1968/I' '1996/I' '2000/I' '2009/I' '2003/I']


In [374]:
column_name = 'mYear'

# Extract numeric portion using regular expression
titles_df[column_name] = titles_df[column_name].str.extract('(\d+)', expand=False)
titles_df[column_name] = titles_df[column_name].astype(int)


In [375]:
titles_df.head()

,mId,mName,mYear,mRating,mVotes,mGenre
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227.0,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289.0,['action' 'comedy' 'crime' 'drama' 'thriller']


Movie Lines

In [376]:
column_names = ['lId', 'chId', 'mId', 'chName', 'chLine']
lines_df = pd.read_csv('movie_lines.tsv', sep='\t', header=None,names=column_names, on_bad_lines='skip')
lines_df.head()

,lId,chId,mId,chName,chLine
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [377]:
print(lines_df.shape)
print(lines_df.info())

(293202, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293202 entries, 0 to 293201
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lId     293202 non-null  object
 1   chId    288917 non-null  object
 2   mId     288917 non-null  object
 3   chName  288874 non-null  object
 4   chLine  288663 non-null  object
dtypes: object(5)
memory usage: 11.2+ MB
None


In [378]:
nan_percentage = (lines_df.isna().mean() * 100).round(2)
print(nan_percentage)

lId       0.00
chId      1.46
mId       1.46
chName    1.48
chLine    1.55
dtype: float64


In [379]:
lines_df = lines_df.dropna()

In [380]:
# Remove duplicate rows
duplicate_counts = lines_df.duplicated().sum()
print("Total duplicates: ",duplicate_counts)
if duplicate_counts > 0:
  lines_df = lines_df.drop_duplicates()

Total duplicates:  0


In [381]:
print(lines_df.shape)

(288620, 5)


In [382]:
column_names = ['chId1', 'chId2','mId', 'lineList']
conversations_df = pd.read_csv('movie_conversations.tsv', sep='\t',names=column_names, header=None, on_bad_lines='skip')
conversations_df.head()

,chId1,chId2,mId,lineList
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']


In [383]:
nan_percentage = (conversations_df.isna().mean() * 100).round(2)
print(nan_percentage)

chId1       0.0
chId2       0.0
mId         0.0
lineList    0.0
dtype: float64


In [384]:
# Remove duplicate rows
duplicate_counts = lines_df.duplicated().sum()
print("Total Number Of Duplicates: ", duplicate_counts)
if duplicate_counts > 0:
  lines_df = lines_df.drop_duplicates()

Total Number Of Duplicates:  0


In [385]:
conversations_df.shape

(83097, 4)

In [386]:
conversations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83097 entries, 0 to 83096
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   chId1     83097 non-null  object
 1   chId2     83097 non-null  object
 2   mId       83097 non-null  object
 3   lineList  83097 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB


In [387]:
column_names = ['mId','mName','url']
rawScript_df = pd.read_csv('raw_script_urls.tsv', sep='\t', names=column_names, header=None,on_bad_lines='skip')
rawScript_df.head()

,mId,mName,url
0,m0,10 things i hate about you,http://www.dailyscript.com/scripts/10Things.html
1,m1,1492: conquest of paradise,http://www.hundland.org/scripts/1492-ConquestO...
2,m2,15 minutes,http://www.dailyscript.com/scripts/15minutes.html
3,m3,2001: a space odyssey,http://www.scifiscripts.com/scripts/2001.txt
4,m4,48 hrs.,http://www.awesomefilm.com/script/48hours.txt


In [388]:
rawScript_df.shape

(617, 3)

In [389]:
nan_percentage = (rawScript_df.isna().mean() * 100).round(2)
print(nan_percentage)

mId      0.00
mName    0.16
url      0.16
dtype: float64


In [390]:
# Remove rows with NaN values
rawScript_df = rawScript_df.dropna()

In [391]:
# Remove duplicate rows
duplicate_counts = rawScript_df.duplicated().sum()
print("Total Number Of Duplicates: ", duplicate_counts)
if duplicate_counts > 0:
  rawScript_df = rawScript_df.drop_duplicates()

Total Number Of Duplicates:  0


In [392]:
rawScript_df.shape

(616, 3)

Here are the list of initial dataframes:
1. characters_df
2. titles_df
3. conversations_df
4. rawScript_df
5. lines_df

In [393]:
unique_movies_count = characters_df['mId'].nunique()
print("Number of unique movies:", unique_movies_count)

Number of unique movies: 569


In [394]:
# Movies in titles_df but not in characters_df
movies_in_titles_not_in_characters = set(titles_df['mId']) - set(characters_df['mId'])

# Movies in characters_df but not in titles_df
movies_in_characters_not_in_titles = set(characters_df['mId']) - set(titles_df['mId'])

# Convert the sets to lists for easier handling or further analysis
movies_in_titles_not_in_characters_list = list(movies_in_titles_not_in_characters)
movies_in_characters_not_in_titles_list = list(movies_in_characters_not_in_titles)

print("Movies in titles_df but not in characters_df:", movies_in_titles_not_in_characters_list)
print("Movies in characters_df but not in titles_df:", movies_in_characters_not_in_titles_list)


Movies in titles_df but not in characters_df: ['m321', 'm588', 'm75', 'm457', 'm291', 'm600', 'm456', 'm412', 'm337', 'm273', 'm96', 'm338', 'm48', 'm602', 'm181', 'm614', 'm69', 'm364', 'm275', 'm593', 'm314', 'm120', 'm475', 'm565', 'm603', 'm270', 'm134', 'm388', 'm307', 'm115', 'm483', 'm406', 'm237', 'm236', 'm605', 'm430', 'm135', 'm536', 'm488', 'm308', 'm521', 'm484', 'm616', 'm251', 'm160', 'm535', 'm64']
Movies in characters_df but not in titles_df: []


In [395]:
detailed_characters_df = characters_df.merge(titles_df[['mId', 'mYear', 'mRating', 'mVotes', 'mGenre']], on='mId', how='left')
print("Dimension Of Charaters Dataframe",characters_df.shape)
print("Dimension Of Merged Character Dataframe",detailed_characters_df.shape)
detailed_characters_df.head()

Dimension Of Charaters Dataframe (2679, 6)
Dimension Of Merged Character Dataframe (2679, 10)


,chId,chName,mId,mName,gender,posCredits,mYear,mRating,mVotes,mGenre
0,u0,BIANCA,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance']
1,u2,CAMERON,m0,10 things i hate about you,M,3,1999,6.9,62847.0,['comedy' 'romance']
2,u4,JOEY,m0,10 things i hate about you,M,6,1999,6.9,62847.0,['comedy' 'romance']
3,u5,KAT,m0,10 things i hate about you,F,2,1999,6.9,62847.0,['comedy' 'romance']
4,u6,MANDELLA,m0,10 things i hate about you,F,7,1999,6.9,62847.0,['comedy' 'romance']


In [396]:
duplicate_counts = detailed_characters_df.duplicated().sum()
print("Total Number Of Duplicates: ", duplicate_counts)

Total Number Of Duplicates:  0


In [397]:
nan_percentage = (detailed_characters_df.isna().mean() * 100).round(2)
print(nan_percentage)

chId          0.0
chName        0.0
mId           0.0
mName         0.0
gender        0.0
posCredits    0.0
mYear         0.0
mRating       0.0
mVotes        0.0
mGenre        0.0
dtype: float64


In [398]:
# Characters in lines_df but not in detailed_characters_df
characters_in_lines_not_in_merged = set(lines_df['chId']) - set(detailed_characters_df['chId'])

# Characters in merged_df but not in lines_df
characters_in_merged_not_in_lines = set(detailed_characters_df['chId']) - set(lines_df['chId'])

# Convert the sets to lists for easier handling or further analysis
characters_in_lines_not_in_merged_list = list(characters_in_lines_not_in_merged)
characters_in_merged_not_in_lines_list = list(characters_in_merged_not_in_lines)

print("Characters in lines_df but not in detailed_characters_df:", characters_in_lines_not_in_merged_list)
print("Characters in detailed_characters_df but not in lines_df:", characters_in_merged_not_in_lines_list)


Characters in lines_df but not in detailed_characters_df: ['u4377', 'u14', 'u1560', 'u2631', 'u4767', 'u133', 'u3827', 'u211', 'u8478', 'u4308', 'u3577', 'u458', 'u5762', 'u7487', 'u7444', 'u4813', 'u2896', 'u8691', 'u491', 'u5046', 'u2676', 'u2468', 'u8920', 'u2239', 'u8031', 'u5578', 'u3741', 'u2811', 'u7567', 'u995', 'u6120', 'u3554', 'u4482', 'u4825', 'u473', 'u3841', 'u1465', 'u6753', 'u3244', 'u627', 'u7140', 'u4798', 'u844', 'u1246', 'u5436', 'u8518', 'u4842', 'u5317', 'u368', 'u2484', 'u8459', 'u8294', 'u6472', 'u4793', 'u4729', 'u584', 'u7271', 'u6058', 'u2831', 'u8363', 'u8795', 'u8692', 'u146', 'u53', 'u6645', 'u4338', 'u3350', 'u5324', 'u7153', 'u2093', 'u7317', 'u7902', 'u2379', 'u4228', 'u4775', 'u8773', 'u4904', 'u577', 'u6743', 'u5645', 'u40', 'u187', 'u4206', 'u5533', 'u940', 'u9005', 'u4672', 'u6869', 'u7286', 'u4026', 'u8852', 'u3492', 'u2278', 'u561', 'u845', 'u8982', 'u2076', 'u4265', 'u762', 'u882', 'u8599', 'u903', 'u6478', 'u1403', 'u7090', 'u2007', 'u325', 'u39

In [399]:
##When we remove the lId we may see charaters repeating dialogues  
lines_df_mod = lines_df[['chId', 'mId', 'chName', 'chLine']]

# Remove duplicate rows
duplicate_counts = lines_df_mod.duplicated().sum()
print("Total Number Of Duplicates: ", duplicate_counts)
if duplicate_counts > 0:
   lines_df_mod = lines_df_mod.drop_duplicates()
   
lines_df_mod.shape

Total Number Of Duplicates:  5438


(283182, 4)

In [400]:
df = detailed_characters_df.merge(lines_df_mod, on=['chId', 'mId'], how='inner')
df.head()

,chId,chName_x,mId,mName,gender,posCredits,mYear,mRating,mVotes,mGenre,chName_y,chLine
0,u0,BIANCA,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],BIANCA,They do not!
1,u0,BIANCA,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],BIANCA,I hope so.
2,u0,BIANCA,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],BIANCA,Let's go.
3,u0,BIANCA,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],BIANCA,Okay -- you're gonna need to learn how to lie.
4,u0,BIANCA,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],BIANCA,Like my fear of wearing pastels?


In [401]:
# Filter rows where chName_x does not match chName_y
mismatched_names_df = df[df['chName_x'] != df['chName_y']]

# Display the rows with mismatched names
print("Rows with mismatched chName_x and chName_y:")
print(mismatched_names_df)

Rows with mismatched chName_x and chName_y:
Empty DataFrame
Columns: [chId, chName_x, mId, mName, gender, posCredits, mYear, mRating, mVotes, mGenre, chName_y, chLine]
Index: []


In [402]:
df['chName'] = df['chName_x']
df = df.drop(['chName_y','chName_x'], axis=1)
df.head()

,chId,mId,mName,gender,posCredits,mYear,mRating,mVotes,mGenre,chLine,chName
0,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],They do not!,BIANCA
1,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],I hope so.,BIANCA
2,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],Let's go.,BIANCA
3,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],Okay -- you're gonna need to learn how to lie.,BIANCA
4,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],Like my fear of wearing pastels?,BIANCA


In [403]:
nan_percentage = (df.isna().mean() * 100).round(2)
print(nan_percentage)

chId          0.0
mId           0.0
mName         0.0
gender        0.0
posCredits    0.0
mYear         0.0
mRating       0.0
mVotes        0.0
mGenre        0.0
chLine        0.0
chName        0.0
dtype: float64


In [404]:
# Remove duplicate rows
duplicate_counts = df.duplicated().sum()
print("Total Number Of Duplicates: ", duplicate_counts)
if duplicate_counts > 0:
   df = df.drop_duplicates()
df.shape

Total Number Of Duplicates:  0


(200352, 11)

In [405]:
##Get the number of unique dialogues spoken by each characters.
df['dialoguesCount'] = df.groupby(['chId', 'mId'])['chLine'].transform('count')

##Get the number of characters per dialogue
df['charCountPerDialogue'] = df.chLine.str.len()

##Get the number of words per dialogue
df['wordCountPerDialogue'] = df.chLine.str.count(' ') + 1

In [406]:
df.head()

,chId,mId,mName,gender,posCredits,mYear,mRating,mVotes,mGenre,chLine,chName,dialoguesCount,charCountPerDialogue,wordCountPerDialogue
0,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],They do not!,BIANCA,92,12,3
1,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],I hope so.,BIANCA,92,10,3
2,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],Let's go.,BIANCA,92,9,2
3,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],Okay -- you're gonna need to learn how to lie.,BIANCA,92,46,10
4,u0,m0,10 things i hate about you,F,4,1999,6.9,62847.0,['comedy' 'romance'],Like my fear of wearing pastels?,BIANCA,92,32,6
